In [1]:
%run network.ipynb

Looking in links: https://download.pytorch.org/whl/torch_stable.html



In [2]:
%run data_loader.ipynb

In [3]:
%run utils.ipynb

In [4]:
import torch.nn as nn

class CrowdCounter(nn.Module):
    def __init__(self):
        super(CrowdCounter, self).__init__()        
        self.DME = MCNN()        
        self.loss_fn = nn.MSELoss()
        
    @property
    def loss(self):
        return self.loss_mse
    
    def forward(self,  im_data, gt_data=None):        
        im_data = np_to_variable(im_data, is_cuda=True, is_training=self.training)   
        density_map = self.DME(im_data)
        
        if self.training:                        
            gt_data = np_to_variable(gt_data, is_cuda=True, is_training=self.training)            
            self.loss_mse = self.build_loss(density_map, gt_data)
            
        return density_map
    
    def build_loss(self, density_map, gt_data):
        loss = self.loss_fn(density_map, gt_data)
        return loss

In [5]:
def evaluate_model(trained_model, data_loader):
    net = CrowdCounter()
    load_net(trained_model, net)
    net.cuda()
    net.eval()
    mae = 0.0
    mse = 0.0
    for blob in data_loader:                        
        im_data = blob['data'].numpy()
        im_data = np.expand_dims(im_data, 1) 
        gt_data = blob['gt_density'].numpy()
        gt_data = np.expand_dims(gt_data, 1)
        density_map = net(im_data, gt_data)
        density_map = density_map.data.cpu().numpy()
        gt_count = np.sum(gt_data)
        et_count = np.sum(density_map)
        mae += abs(gt_count-et_count)
        mse += ((gt_count-et_count)*(gt_count-et_count))        
    mae = mae/len(data_loader)
    mse = np.sqrt(mse/len(data_loader))
    return mae,mse

In [6]:
output_dir = '../saved_models_new/'
train_path = '../ShanghaiTech/ShanghaiTech/part_B/train_data/images/'
train_gt_path = '../ShanghaiTech/ShanghaiTech/part_B/train_data/ground-truth-h5/'
val_path = '../ShanghaiTech/ShanghaiTech/part_B/train_data/val/'
val_gt_path = '../ShanghaiTech/ShanghaiTech/part_B/train_data/val-den/'

image_paths = os.listdir(train_path)

index = -1
for i in range(len(image_paths)):
    if image_paths[i].find('.jpg') >= 0:
        image_paths[i] = train_path + image_paths[i]
    else:
        index = i
        
del image_paths[index]

target_paths = os.listdir(train_gt_path)
for i in range(len(target_paths)):
    if target_paths[i].find('.h5') >= 0:
        target_paths[i] = train_gt_path + target_paths[i]
    else:
        index = i
        
del target_paths[index]
    
image_paths_val = os.listdir(val_path)
for i in range(len(image_paths_val)):
    image_paths_val[i] = val_path + image_paths_val[i]

target_paths_val = os.listdir(val_gt_path)

for i in range(len(target_paths_val)):
    target_paths_val[i] = val_gt_path + target_paths_val[i]



In [7]:
#data_loader = ImageDataLoader(train_path, train_gt_path, shuffle=True, gt_downsample=True, pre_load=True)
#data_loader_val = ImageDataLoader(val_path, val_gt_path, shuffle=False, gt_downsample=True, pre_load=True)

transform = transforms.Compose(
    [
    #transforms.Resize((256, 172), interpolation=2),
    transforms.ToTensor(),
     #transforms.CenterCrop(10),
    #transforms.Normalize((.5, .5), (.5, .5))
    ]
)

trainset = DatasetLoaderH5(image_paths, target_paths)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)

trainset_val = DatasetLoaderH5(image_paths_val, target_paths_val)
trainloader_val = torch.utils.data.DataLoader(trainset_val, batch_size=20, shuffle=False, num_workers=0)

data_loader = iter(trainloader)
data_loader_val = iter(trainloader_val)


In [8]:
start_step = 0
end_step = 500
lr = 0.00001
momentum = 0.9
disp_interval = 4
log_interval = 250

In [9]:
rand_seed = 64678  
if rand_seed is not None:
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)
    torch.cuda.manual_seed(rand_seed)


# load net
net = CrowdCounter()
weights_normal_init(net, dev=0.01)
net.cuda()
net.train()

params = list(net.parameters())
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [10]:
import sys
from tqdm import tqdm

# training
train_loss = 0
step_cnt = 0
best_mae = sys.maxsize * 2 + 1
method = 'MCNN'
dataset_name = 'shtechB'

for epoch in range(start_step, end_step+1):    
    step = -1
    train_loss = 0
    run = tqdm(enumerate(data_loader, 0))
    for i, blob in run:                
        step += 1
        im_data = blob['data'].numpy()
        im_data = np.expand_dims(im_data, 1) 
        gt_data = blob['gt_density'].numpy()
        gt_data = np.expand_dims(gt_data, 1)
        #print(np.sum(gt_data))
        density_map = net(im_data, gt_data)
        loss = net.loss
        train_loss += loss.data
        step_cnt += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % disp_interval == 0:            
            gt_count = np.sum(gt_data)
            density_map = density_map.data.cpu().numpy()
            et_count = np.sum(density_map)
            #print('Crowd count(et, gt):', et_count, gt_count)
            save_results(im_data,gt_data,density_map, output_dir)
            
        run.set_description("Train loss: {}".format(train_loss))
        run.refresh() # to show immediately the update
    
    data_loader = iter(trainloader)
    if (epoch % 2 == 0):
        save_name = os.path.join(output_dir, '{}_{}_{}.h5'.format(method,dataset_name,epoch))
        save_net(save_name, net)     
        #calculate error on the validation dataset 
        mae,mse = evaluate_model(save_name, data_loader_val)
        data_loader_val = iter(trainloader_val)
        if mae < best_mae:
            best_mae = mae
            best_mse = mse
            best_model = '{}_{}_{}.h5'.format(method,dataset_name,epoch)
        log_text = 'EPOCH: %d, MAE: %.1f, MSE: %0.1f' % (epoch,mae,mse)
        print(log_text)
        log_text = 'BEST MAE: %0.1f, BEST MSE: %0.1f, BEST MODEL: %s' % (best_mae,best_mse, best_model)
        print(log_text)

Train loss: 0.0003521870239637792: : 360it [00:13, 27.02it/s] 
Train loss: 1.3222420420788694e-06: : 3it [00:00, 26.29it/s]

EPOCH: 0, MAE: 387.4, MSE: 398.8
BEST MAE: 387.4, BEST MSE: 398.8, BEST MODEL: MCNN_shtechB_0.h5


Train loss: 0.00034226838033646345: : 360it [00:12, 28.05it/s]
Train loss: 0.0003393283986952156: : 360it [00:12, 27.98it/s] 
Train loss: 2.479318709447398e-06: : 3it [00:00, 28.55it/s] 

EPOCH: 2, MAE: 363.9, MSE: 371.7
BEST MAE: 363.9, BEST MSE: 371.7, BEST MODEL: MCNN_shtechB_2.h5


Train loss: 0.0003376800159458071: : 360it [00:12, 28.34it/s] 
Train loss: 0.00033502475707791746: : 360it [00:12, 28.35it/s]
Train loss: 1.5090173292264808e-05: : 3it [00:00, 27.75it/s]

...

EPOCH: 320, MAE: 645.2, MSE: 645.3
BEST MAE: 47.5, BEST MSE: 60.0, BEST MODEL: MCNN_shtechB_34.h5


Train loss: 0.00019610147865023464: : 360it [00:12, 28.23it/s]
Train loss: 0.0002013265184359625: : 360it [00:12, 28.22it/s] 
Train loss: 4.645297337901866e-07: : 3it [00:00, 27.50it/s]

EPOCH: 322, MAE: 666.0, MSE: 666.0
BEST MAE: 47.5, BEST MSE: 60.0, BEST MODEL: MCNN_shtechB_34.h5


Train loss: 0.00013852912525180727: : 239it [00:08, 28.31it/s]


KeyboardInterrupt: 